# Infomap

In [1]:
import igraph
import datetime
import sys
import os
import csv

# Saved the dataframe after added the "Tweet_From" column and reading the file here.
data_language_german = pd.read_csv("colab_3Dec.csv")

ImportError: DLL load failed: The specified module could not be found.

In [ ]:
graph_name = "test.gml"  #File path and name to save the gml file
edges = [] 
nodes = set()
tf_from = [tf for tf in data_language_german["Tweet_From"]]
edges = zip(data_language_german['Tweet_From'],data_language_german['Screen Name'])
#edges += [(follower, username) for follower in tf_from]
nodes.update(tf_from)
edges = list(set(tuple(sorted(edge)) for edge in edges))
nodes = list(set(nodes))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
nx.write_gml(G, graph_name)

In [ ]:
DATA_DIR = r'\\'
def save(g, clusters, filename):
    print('Saving...', filename)
    membership = clusters.membership
    writer = csv.writer(open(DATA_DIR+filename, "w"))
    writer.writerow(['Id', 'Label', 'Modularity Class'])
    for id, name, memb in zip(g.vs["id"], g.vs["label"], membership):
        writer.writerow([id, name, memb])


graph_file_name = r'C:\Users\RR510300\Downloads\block\30_Nov_2021\test\test.gml' ## Read the .gml file
g = igraph.read(graph_file_name)

lp_csv = "lp.csv"
lp = g.community_label_propagation()  # Using Community label propagation
save(g, lp, lp_csv)

ml_csv = "ml.csv"
ml = g.community_multilevel()  # Using Community multilevel
save(g, ml, ml_csv)

im_csv = 'im.csv'
im = g.community_infomap()     #Using Infomap
save(g, im, im_csv)

# Network analysis using betweenness_centrality

In [ ]:
# This script works on 2 columns in the tweets file received through the Twitter API: " screen_name" and " text" (note the blankspaces inserted by Twitter)
# To make it work with the output of GetOldTweets3, change the label username to screenname with a leading blankspace, and add a leading blankspace to the label text
# Line 94 has the input file name tweets.csv (change it as needed)
# Output file network_edges.csv has two columns.
# The first (left) column is the target and the second (right) column is the source in a directed network
import pandas as pd
import numpy as np
import os

#Definition for strippping whitespace
def trim(dataset):
    trim = lambda x: x.strip() if type(x) is str else x
    return dataset.applymap(trim)

def construct_edges(tweeters):
    """
    :param tweeters: a datafrmae of twitter handles
    :return: A dataframe of edges with 'handles of retweeted and reply/mentions' as first column
    and the twitter of handle of the tweeted person as the second column.
    """
    network_edges = []
    for idx, row in tweeters.iterrows():
        for name in list(np.array(row.rt_name).flat):
            network_edges.append({'rt_name': row['Screen Name'], ' screen_name': name })
    network_edges = pd.DataFrame(network_edges)
    return network_edges


def get_rt_names(tweets):
  
    import re

    def get_names(tweet):
 
        pattern = re.compile('^@.*')
        text = tweet['Tweet Content']
        text_tokens = str(text).split()
        screen_name = tweet['Screen Name']
        matches = []
        for tok in text_tokens:
            match_name = pattern.match(tok)
            if (match_name):
                rt_name = match_name.group()
                rt_name = re.sub('[^\w]', "", rt_name)
                matches.append(rt_name)
        if not matches:
            matches.append(screen_name)
        return matches

    if 'Screen Name' in tweets.columns:
        if 'Tweet Content' in tweets.columns:
            tweets['rt_name'] = tweets.apply(get_names, axis=1)
        else:
            print("please check for the column names")
            print(" 'Tweet Content': column name for the tweet text is missing")
            print(" you might check for extra white spaces as well")
    else:
        print("please check for the column names")
        print(" ' screen_name': column name for the twitter profle name is missing")
        print(" you might check for extra white spaces as well")
    tweeters = tweets[[ 'rt_name', 'Screen Name']]
    return tweeters


def read_csv():
    """
    :return: tweets file in latin1 encoding.
    """
    try:
        tweets = pd.read_csv(r"C:\Users\RR510300\Downloads\block\30_Nov_2021\test\colab_3Dec.csv", encoding='latin1')

    except:
        print("File not found")
        print("Please check if the file named 'tweets.csv' exists in the directory:")
        print(os.getcwd() + "/")
        exit()
    return tweets


def main():
    """
    :tweeters: two column file with screen name of tweet and list of retweet names for each tweet
    :network_edges: created by taking tweeters and turning it into edgelist
    :network_edges.to_csv: outputs dataframe to file
    """
    from datetime import datetime
    tweets = read_csv()
    if tweets is None:
        return
    screen_name_list = []
    rt_name_list = []
    tweeters = get_rt_names(tweets)
    for row in range(0,len(tweeters['Screen Name'])):
        screen_name = tweeters['Screen Name'][row]
        for retweet in tweeters['rt_name'][row]:
            screen_name_list.append(screen_name)
            rt_name_list.append(retweet)     
    network_edges = pd.DataFrame()
    network_edges['rt_name'] = rt_name_list
    network_edges['Screen Name'] = screen_name_list
    # file_name = str(os.getcwd()) + "\\" + "network_edges" + "_" + str(datetime.now()) + ".csv"
    network_edges.to_csv("output.csv", index=False)
    print("edges file is ready @: " + str(os.getcwd()))

# compile the file and run the function below.
main()

In [ ]:
def from_pandas_edgelist(
    df,
    source="source",
    target="target",
    edge_attr=None,
    create_using=None,
    edge_key=None,
):
    """Returns a graph from Pandas DataFrame containing an edge list.

    The Pandas DataFrame should contain at least two columns of node names and
    zero or more columns of edge attributes. Each row will be processed as one
    edge instance.

    Note: This function iterates over DataFrame.values, which is not
    guaranteed to retain the data type across columns in the row. This is only
    a problem if your row is entirely numeric and a mix of ints and floats. In
    that case, all values will be returned as floats. See the
    DataFrame.iterrows documentation for an example.

    Parameters
    ----------
    df : Pandas DataFrame
        An edge list representation of a graph

    source : str or int
        A valid column name (string or integer) for the source nodes (for the
        directed case).

    target : str or int
        A valid column name (string or integer) for the target nodes (for the
        directed case).

    edge_attr : str or int, iterable, True, or None
        A valid column name (str or int) or iterable of column names that are
        used to retrieve items and add them to the graph as edge attributes.
        If `True`, all of the remaining columns will be added.
        If `None`, no edge attributes are added to the graph.

    create_using : NetworkX graph constructor, optional (default=nx.Graph)
        Graph type to create. If graph instance, then cleared before populated.

    edge_key : str or None, optional (default=None)
        A valid column name for the edge keys (for a MultiGraph). The values in
        this column are used for the edge keys when adding edges if create_using
        is a multigraph.

    See Also
    --------
    to_pandas_edgelist

    Examples
    --------
    Simple integer weights on edges:

    >>> import pandas as pd
    >>> pd.options.display.max_columns = 20
    >>> import numpy as np
    >>> rng = np.random.RandomState(seed=5)
    >>> ints = rng.randint(1, 11, size=(3, 2))
    >>> a = ["A", "B", "C"]
    >>> b = ["D", "A", "E"]
    >>> df = pd.DataFrame(ints, columns=["weight", "cost"])
    >>> df[0] = a
    >>> df["b"] = b
    >>> df[["weight", "cost", 0, "b"]]
       weight  cost  0  b
    0       4     7  A  D
    1       7     1  B  A
    2      10     9  C  E
    >>> G = nx.from_pandas_edgelist(df, 0, "b", ["weight", "cost"])
    >>> G["E"]["C"]["weight"]
    10
    >>> G["E"]["C"]["cost"]
    9
    >>> edges = pd.DataFrame(
    ...     {
    ...         "source": [0, 1, 2],
    ...         "target": [2, 2, 3],
    ...         "weight": [3, 4, 5],
    ...         "color": ["red", "blue", "blue"],
    ...     }
    ... )
    >>> G = nx.from_pandas_edgelist(edges, edge_attr=True)
    >>> G[0][2]["color"]
    'red'

    Build multigraph with custom keys:

    >>> edges = pd.DataFrame(
    ...     {
    ...         "source": [0, 1, 2, 0],
    ...         "target": [2, 2, 3, 2],
    ...         "my_edge_key": ["A", "B", "C", "D"],
    ...         "weight": [3, 4, 5, 6],
    ...         "color": ["red", "blue", "blue", "blue"],
    ...     }
    ... )
    >>> G = nx.from_pandas_edgelist(
    ...     edges,
    ...     edge_key="my_edge_key",
    ...     edge_attr=["weight", "color"],
    ...     create_using=nx.MultiGraph(),
    ... )
    >>> G[0][2]
    AtlasView({'A': {'weight': 3, 'color': 'red'}, 'D': {'weight': 6, 'color': 'blue'}})


    """
    g = nx.empty_graph(0, create_using)

    if edge_attr is None:
        g.add_edges_from(zip(df[source], df[target]))
        return g

    reserved_columns = [source, target]

    # Additional columns requested
    attr_col_headings = []
    attribute_data = []
    if edge_attr is True:
        attr_col_headings = [c for c in df.columns if c not in reserved_columns]
    elif isinstance(edge_attr, (list, tuple)):
        attr_col_headings = edge_attr
    else:
        attr_col_headings = [edge_attr]
    if len(attr_col_headings) == 0:
        raise nx.NetworkXError(
            f"Invalid edge_attr argument: No columns found with name: {attr_col_headings}"
        )

    try:
        attribute_data = zip(*[df[col] for col in attr_col_headings])
    except (KeyError, TypeError) as e:
        msg = f"Invalid edge_attr argument: {edge_attr}"
        raise nx.NetworkXError(msg) from e

    if g.is_multigraph():
        # => append the edge keys from the df to the bundled data
        if edge_key is not None:
            try:
                multigraph_edge_keys = df[edge_key]
                attribute_data = zip(attribute_data, multigraph_edge_keys)
            except (KeyError, TypeError) as e:
                msg = f"Invalid edge_key argument: {edge_key}"
                raise nx.NetworkXError(msg) from e

        for s, t, attrs in zip(df[source], df[target], attribute_data):
            if edge_key is not None:
                attrs, multigraph_edge_key = attrs
                key = g.add_edge(s, t, key=multigraph_edge_key)
            else:
                key = g.add_edge(s, t)

            g[s][t][key].update(zip(attr_col_headings, attrs))
    else:
        for s, t, attrs in zip(df[source], df[target], attribute_data):
            g.add_edge(s, t)
            g[s][t].update(zip(attr_col_headings, attrs))

    return g

In [ ]:
#import statements
import networkx as nx
import pandas as pd
import numpy as np
import math

df = pd.read_csv('output.csv') #read in CSV file with edgelist
df.columns = ["source", "target"]
G = from_pandas_edgelist(df, create_using=nx.DiGraph()) #create graph from edgelist as a directed network (DiGraph())

Indegrees = [val for val in G.in_degree().values()] #gets the in-degree of all nodes in the network
Outdegrees = [val for  val in G.out_degree().values()] #gets the out-degree of all nodes in the network
Nodes = G.nodes() #gets the name of all nodes in the network

G = from_pandas_edgelist(df, create_using=nx.Graph()) #recreate the network as a undirected network to calculate betweenness score and plot

Betweenness_dict = nx.betweenness_centrality(G, normalized=False) #gets the betweenness score for of all nodes in the network
for node,value in Betweenness_dict.items():
    Betweenness_dict[node] = value/2 #divide each betweenness score by 2 because in directed networks betweeness scores are half what they would be in undirected networks
n_color = np.asarray([(Betweenness_dict[n] + 1) for n in Nodes]) #creates list of users from highest to lowest betweenness score for the color map
#nx.draw(G, node_color=n_color, cmap="cool", node_size=[math.log(v+1/2,10) * 2 for v in Betweenness_dict.values()]) #draws the undirected network using a color map and also adjusts node size by the log of the betweenness score + 1

network_analysis_df = pd.DataFrame() #creates dataframe to output analysis
network_analysis_df["Nodes"] = Nodes #column 1 inlcudes each node name 
network_analysis_df["In-Degree"] = Indegrees #column 2 inlcudes each node's in-degree 
network_analysis_df["Out-Degree"] = Outdegrees #column 3 inlcudes each node's out-degree 
network_analysis_df["Betweenness"] = Betweenness_dict.values() #column 4 inlcudes each node's betweenness score
network_analysis_df.to_excel("output.xlsx") #outputs the dataframe to an excel file